In [6]:
from random import randint

class Cell:
    def __init__(self):
        self.value = 0
        
    def __bool__(self):
        return self.value == 0

class TicTacToe:
    FREE_CELL = 0      # свободная клетка
    HUMAN_X = 1        # крестик (игрок - человек)
    COMPUTER_O = 2     # нолик (игрок - компьютер)
    
    def __init__(self):
        self.init()
        
    def clear(self):
        self.pole = tuple(tuple(Cell() for _ in '123')for _ in '123')
    
    def __getitem__(self, indx):
        try:
            r, c = indx
            if isinstance(c, slice):
                return tuple(obj.value for obj in self.pole[r][c])
            if isinstance(r, slice):
                return tuple(obj.value for obj in tuple(zip(*self.pole))[r][c])
            return self.pole[r][c].value
        except:
            raise IndexError('некорректно указанные индексы')
            
    def __setitem__(self, indx, value):
        try:
            r, c = indx
            if self.pole[r][c]:
                self.pole[r][c].value = value
            else:
                raise ValueError('клетка уже занята')
        except:
            raise IndexError('некорректно указанные индексы')
            
    def init(self):
        self.clear()
        self.__second_moves = 8
    
    def show(self):
        for i in range(len(self.pole)):
            print(*self[i,:])
            
    def human_go(self):
        r, c = map(int, input('Введите координаты через пробел').split())
        self[r, c] = self.HUMAN_X
    
    def computer_go(self):
        rnd = randint(0, self.__second_moves - 1)
        res = []
        for row in self.pole:
            for cell in row:
                if cell:
                    res.append(cell)
        self.__second_moves -= 2
        res[rnd].value = self.COMPUTER_O
    
    def is_win(self, who):
        '''суммируются линии и диагонали в отфиальрованом по игроку "who" поле.
            В список "m" добавляются результаты сравнения с заполеной линией "res".
            При любом совпадении возвращается True "any(m)"
            '''
        m = []
        d1 = d2 = 0 
        n = len(self.pole)
        res = who * n
        for i in range(n):
            m.append(sum(x if x == who else 0 for x in self[i,:]) == res)
            m.append(sum(x if x == who else 0 for x in self[:,i]) == res)        
        for i in range(n):
            d1 += who if self[i,i] == who else 0
            d2 += who if self[i, n-i-1] == who else 0
        m.append(d1 == res)
        m.append(d2 == res)
        return any(m)
        
    @property
    def is_human_win(self):
        return self.is_win(self.HUMAN_X)
    
    @property
    def is_computer_win(self):
        return self.is_win(self.COMPUTER_O)
        
    @property
    def is_draw(self): #наличие Cell.value = 0. True, ели нет
        return all(all(bool(x.value) for x in row) for row in self.pole) or (self.is_computer_win and self.is_human_win)
        
    def __bool__(self):
        return not(self.is_computer_win or self.is_human_win or self.is_draw)
    
    

In [2]:
# С ИИ
class TicTacToe:
    FREE_CELL = 0  # свободная клетка
    HUMAN_X = 1  # крестик (игрок - человек)
    COMPUTER_O = 2  # нолик (игрок - компьютер)

    def __init__(self):
        '''Инициализация'''
        self.pole = [[Cell() for _ in range(3)] for _ in range(3)]
        self.free_cells = 9
        self.winner = None
        self.diagonals = [[self.pole[i][i] for i in range(3)], [self.pole[-i-1][i] for i in range(-3, 0)]]
        for diag in self.diagonals:
            for cell in diag:
                cell._priority += 2

    def __getitem__(self, coords):
        '''Геттер значения клетки по координатам'''
        if self._valid_coords(*coords):
            return self.pole[coords[0]][coords[1]].value
        raise IndexError('некорректно указанные индексы')

    def __setitem__(self, coords, value):
        '''Сеттер значения клетки по координатам'''
        if not self._valid_coords(*coords):
            raise IndexError('некорректно указанные индексы')
        if value in (0, 1, 2):
            self.pole[coords[0]][coords[1]].value = value
        self.__bool__()

    def __bool__(self):
        '''Проверка окончена игра или нет'''
        for row in self.pole:
            if not any(row) and len(set(row)) == 1:
                self.winner = row[0].value
                return False
        for col in zip(*self.pole):
            if not any(col) and len(set(col)) == 1:
                self.winner = col[0].value
                return False
        for diag in self.diagonals:
            if not any(diag) and len(set(diag)) == 1:
                self.winner = diag[0].value
                return False
        if self.free_cells == 0:
            self.winner = 0
            return False
        return True

    @property
    def is_human_win(self):
        return self.winner == self.HUMAN_X

    @property
    def is_computer_win(self):
        return self.winner == self.COMPUTER_O

    @property
    def is_draw(self):
        return self.winner == 0

    @classmethod
    def _valid_coords(cls, y, x):
        '''Проверка координат'''
        return type(y) == type(x) == int and 0 <= y < 3 and 0 <= x < 3

    def init(self):
        self.__init__()

    def human_go(self):
        '''Ход человека'''
        while True:
            print('Введите координаты свободной клетки через пробел:')
            coords = input()
            if len(coords.split()) != 2:
                print('Было введено неверное количество координат.')
                continue
            try:
                y, x = map(int, coords.split())
                if not self._valid_coords(y, x):
                    print('Введённые координаты вне диапазона поля.')
                    continue
            except ValueError:
                print('Введённые данные не являются координатами.')
                continue
            if self.pole[y][x]:
                self.pole[y][x].value = self.HUMAN_X
                self.free_cells -= 1
                self._compute_priority(y, x)
                break
            else:
                print('Выбранная клетка занята.')
                continue

    def computer_go(self):
        '''Ход компьютера'''
        free_cells = [(y, x) for y, row in enumerate(self.pole) for x, cell in enumerate(row) if cell]
        y, x = max(free_cells, key=lambda c: self.pole[c[0]][c[1]]._priority)
        if y == x == 1:  # Реакция на среднюю клетку
            for cell in (self.pole[0][1], self.pole[1][0], self.pole[1][2], self.pole[2][1]):
                cell._priority += 2
        self.free_cells -= 1
        self.pole[y][x].value = self.COMPUTER_O

    def _compute_priority(self, y, x):
        '''Просчёт хода компьютера'''
        # Реакция на ход противника
        for cell in self.pole[y]:
            cell._priority -= 1
        for cell in list(zip(*self.pole))[x]:
            cell._priority -= 1
        if y == x:
            for cell in self.diagonals[0]:
                cell._priority -= 1
        if y + x == 2:
            for cell in self.diagonals[1]:
                cell._priority -= 1
        # Корректировка опасности
        for row in self.pole + [list(lst) for lst in zip(*self.pole)] + self.diagonals:
            counter = [0, 0, 0]
            for cell in row:
                counter[cell.value] += 1
            if counter[1] == 2:
                for cell in row:
                    cell._priority = 5
            if counter[2] == 2:
                for cell in row:
                    cell._priority = 6

    def show(self):
        '''Отрисовка поля'''
        for row in self.pole:
            for cell in row:
                print(cell, end=' ')
            print()
        print('* ' * 10)


class Cell:
    def __init__(self):
        self.value = 0
        self._priority = 2

    def __bool__(self):
        return not self.value

    def __str__(self):
        if self.value == 1:
            return 'X'
        if self.value == 2:
            return '0'
        return '-'

    def __eq__(self, other):
        return self.value == other.value

    def __hash__(self):
        return hash(self.value)

In [7]:
game = TicTacToe()
game.init()
step_game = 0
while game:
    game.show()

    if step_game % 2 == 0:
        game.human_go()
    else:
        game.computer_go()

    step_game += 1


game.show()

if game.is_human_win:
    print("Поздравляем! Вы победили!")
elif game.is_computer_win:
    print("Все получится, со временем")
else:
    print("Ничья.")

0 0 0
0 0 0
0 0 0


Введите координаты через пробел 1 0


0 0 0
1 0 0
0 0 0
2 0 0
1 0 0
0 0 0


Введите координаты через пробел 1 1


2 0 0
1 1 0
0 0 0
2 2 0
1 1 0
0 0 0


...

2 2 0
1 1 0
0 1 0
2 2 0
1 1 2
0 1 0


Введите координаты через пробел 0 2


2 2 1
1 1 2
0 1 0
2 2 1
1 1 2
0 1 2


Введите координаты через пробел 2 0


2 2 1
1 1 2
1 1 2
Поздравляем! Вы победили!


In [75]:
game = TicTacToe()
game.clear()
game[0, 0] = 0
game[1, 0] = 1
game[2, 0] = 1
game[0, 1] = 2
game[1, 1] = 0
game[2, 1] = 2
game[0, 2] = 1
game[1, 2] = 1
game[2, 2] = 2
# формируется поле:
# 1 0 0
# 2 0 0
# 0 0 0
#game[3, 2] = 2 # генерируется исключение IndexError
#if game[0, 0] == 0:
#    game[0, 0] = 2
#assert game[0, :] == (1, 0, 0) and game[1, :] == (2, 0, 0) and game[:, 0] == (1, 2, 3), "некорректно отработали срезы после вызова метода clear() и присваивания новых значений"
game.show()

AttributeError: 'TicTacToe' object has no attribute 'clear'